In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv1D, Activation, MaxPooling2D, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
import time

In [2]:
from pygameMultibot2 import GameWindow

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
Python version 3.8.10 (default, Nov 26 2021, 20:14:08) 
[GCC 9.3.0]


In [3]:
class Warehouse(Env):
    #metadata = {'render.modes' : ['human']}
    def __init__(self):
        self.pygame = GameWindow(0,5,0)
        self.action_space = Discrete(64)
        self.observation_space = Box(low=0, high=6, shape=(11,11,2))

    def reset(self):
        del self.pygame
        self.pygame = GameWindow(0,5,0)
        obs = self.pygame.observe()
        return obs

    def step(self, action):
        self.pygame.action(action)
        obs = self.pygame.observe()
        reward = self.pygame.evaluate()
        done = self.pygame.is_done()
        # print(obs)
        # time.sleep(0.02)
        return obs, reward, done, {}

    def render(self, mode="human", close=False):
        #time.sleep(0.05)
        self.pygame.view1()

In [4]:
env = Warehouse()

In [5]:
states = env.observation_space.shape
actions = env.action_space.n

In [6]:
env.observation_space.sample()

array([[[1.2816148 , 4.348434  ],
        [0.8466811 , 5.3338513 ],
        [4.799029  , 2.1995282 ],
        [1.56701   , 2.8227284 ],
        [5.2789655 , 0.88725173],
        [5.7114263 , 2.077309  ],
        [5.342884  , 2.448163  ],
        [5.408902  , 1.4364749 ],
        [3.6167743 , 1.6063464 ],
        [4.5195923 , 1.7682765 ],
        [0.18139209, 5.0170603 ]],

       [[4.493688  , 3.4493682 ],
        [3.2898126 , 3.8720558 ],
        [2.3728876 , 0.10472283],
        [1.4123354 , 2.8357337 ],
        [1.0432673 , 3.535502  ],
        [2.0238214 , 5.2661066 ],
        [4.501694  , 3.5871546 ],
        [4.427322  , 3.5345576 ],
        [3.5345268 , 4.402026  ],
        [5.9112997 , 0.76401544],
        [3.449539  , 5.9043984 ]],

       [[2.7919343 , 4.4004135 ],
        [2.266243  , 3.6357484 ],
        [3.173887  , 2.6054668 ],
        [5.934462  , 4.185611  ],
        [0.37518668, 1.732787  ],
        [1.4076844 , 0.76547104],
        [4.5306554 , 0.75471973],
        [1

In [7]:
def build_model():
    model = Sequential()
    model.add(tf.keras.layers.Reshape((11,11,2), input_shape=(1,11,11,2)))
    model.add(Conv2D(32, (3,3),strides=(1, 1),activation='relu'))  # OBSERVATION_SPACE_VALUES = (10, 10, 3) a 10x10 RGB image.
    # model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.2))

    model.add(Conv2D(64, (3,3),strides=(1, 1),activation='relu'))
    model.add(Conv2D(64, (3,3),strides=(1, 1),activation='relu'))
    # model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.2))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(512,activation='relu'))

    model.add(Dense(actions, activation='linear'))  # ACTION_SPACE_SIZE = how many choices (9)
    return model

In [13]:
del model

In [14]:
model = build_model()

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 11, 11, 2)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 32)          608       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 5, 5, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               819712    
_________________________________________________________________
dense_3 (Dense)              (None, 64)               

In [16]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [17]:
def build_agent(model, actions):
#     policy = EpsGreedyQPolicy()
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=500000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10000, target_model_update=1e-2, enable_double_dqn=True)
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=0.001), metrics=['mae'])

2022-03-23 14:33:38.182407: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 14:33:38.182730: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 14:33:38.182956: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 14:33:38.183238: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 14:33:38.183463: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [19]:
dqn.compile(Adam(lr=0.00001), metrics=['mae'])
history=dqn.fit(env, nb_steps=1000000, visualize=True, verbose=1)

Training for 1000000 steps ...
Interval 1 (0 steps performed)


/home/windpeh/.cache/pypoetry/virtualenvs/e004dipproject-4aAxvmzB-py3.8/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2022-03-23 14:33:41.298388: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8100


10000/10000 [==============================] - 91s 9ms/step - reward: -0.9842
10 episodes - episode_reward: -984.214 [-1071.470, -884.370]

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 241s 24ms/step - reward: -1.0257
10 episodes - episode_reward: -1025.673 [-1108.575, -958.735] - loss: 0.509 - mae: 24.454 - mean_q: -24.167 - mean_eps: 0.100

Interval 3 (20000 steps performed)
 5463/10000 [===============>..............] - ETA: 1:46 - reward: -1.0011done, took 459.801 seconds


In [43]:
_ = dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...


KeyboardInterrupt: 

In [27]:
dqn.save_weights('training_weights2/', overwrite=True)

In [49]:
dqn.load_weights('training_weights/collision five parcel three boulder2.h5f')